In [60]:
import fitz  # PyMuPDF
import re
import json

## Expresion Regulars

In [61]:
def extraer_metados_gaceta(text_pdf):
    """Extract metadata of a 'gaceta' of the congress in text format"""
    
    metadatos = {}
    
    # --- Date
    date_pattern = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
    match_date = re.search(date_pattern, text_pdf)
    if match_date:
        months = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6, "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
        day_month = int(match_date.group(2))
        month_num = months[match_date.group(3).lower()]
        anio = int(match_date.group(4))
        metadatos["fecha"] = f"{day_month:02d}/{month_num:02d}/{anio}"
        
    # --- Directors
    pattern_directs = r"(?i)DIRECTORES:\s*(.*?)\s*(?:SECRETARIO|RAMA)"
    match_directs = re.search(pattern_directs, text_pdf)
    if match_directs:
        metadatos["directores"] = match_directs.group(1).strip()
    
    # --- Edition of n pages
    pattern_pages = r"(?i)EDICI[ÓO]N\s+DE\s+(\d+)\s+P[ÁA]GINAS"
    match_pages = re.search(pattern_pages, text_pdf)
    if match_pages:
        metadatos["num_paginas"] = int(match_pages.group(1))
        
    # --- Year (Roman Numerals)
    pattern_anio = r"AÑO\s+(M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))"
    match_anio = re.search(pattern_anio, text_pdf)
    if match_anio:
        romano = match_anio.group(1)
        metadatos["Numero_romano"] = romano
        metadatos["Numero_entero"] = whole_number(romano)
        
    # --- Main name
    pattern_name = r"(?i)GACETA\s+DEL\s+CONGRESO"
    if re.search(pattern_name, text_pdf):
        metadatos["nombre"] = "Gaceta del congreso"
    
    # --- ISSN
    pattern_issn = r"(?i)ISSN\s+(\d{4}-\d{3}[\dX])"
    match_issn = re.search(pattern_issn, text_pdf)
    if match_issn:
        metadatos["issn"] = match_issn.group(1)
        
    # --- Legislative branch
    pattern_branch = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+P[ÚU]BLICO"
    if re.search(pattern_branch, text_pdf):
        metadatos["rama"] = "Rama Legislativa del poder publico"
    
    # --- Chamber or Senate
    pattern_entity = r"(?i)(C[ÁA]MARA\s+DE\s+REPRESENTANTES|SENADO\s+DE\s+LA\s+REP[ÚU]BLICA)"
    match_entity = re.search(pattern_entity, text_pdf)
    if match_entity:
        metadatos["camara"] = match_entity.group(1)
    
    # --- Document type
    pattern_type = r"^(PONENCIA|ACTA|PROYECTO\s+DE\s+LEY| INFORME|RESOLUCI[ÓO]N|CONCEPTO|PROPOSICI[ÓO]N|CONSTANCIA|OBJECCIONES)(.*)"
    for linea in text_pdf.splitlines():  # Process line by line
        match_type = re.search(pattern_type, linea, re.IGNORECASE)
        if match_type:
            metadatos["tipo_documento"] = match_type.group(1).strip()
            metadatos["descripcion"] = match_type.group(2).strip()
            break
    
    # --- Presentation
    pattern_presentation = r"(?i)PRESENTACI[ÓO]N\s*:\s*(.*?)\s*(?:DIRECTORES|RAMA|EDICI[ÓO]N)"
    match_presentation = re.search(pattern_presentation, text_pdf)
    if match_presentation:
        metadatos["presentacion"] = match_presentation.group(1).strip()
    
    # Convert the metadata dictionary to a JSON string
    metadatos_json = json.dumps(metadatos, ensure_ascii=False, indent=4)
    return metadatos_json

In [62]:
def whole_number(romano):
    """Convert Roman numeral to whole number"""
    values = {"M": 1000, 'CM': 900, 'D': 500, 'C': 100, 'XC': 90, 'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entire = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in values:
            entire += values[romano[i:i+2]]
            i += 2
        else:
            entire += values[romano[i]]
            i += 1
    return entire

In [63]:
def leer_pdf(ruta_pdf):
    """Read a PDF file and return its text content"""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text()
    return texto


In [64]:

# Ruta del documento PDF
ruta_pdf = "c:/Users/Jorge/OneDrive/Documents/proyect/document/gaceta_171.pdf"

# Leer el contenido del PDF
texto_pdf = leer_pdf(ruta_pdf)

# Extraer metadatos del contenido del PDF
metadatos_json = extraer_metados_gaceta(texto_pdf)
print(metadatos_json)

{
    "fecha": "25/02/2025",
    "directores": "JAIME LUIS LACOUTURE PEÑALOZA",
    "num_paginas": 9,
    "Numero_romano": "XXXIV",
    "Numero_entero": 34,
    "nombre": "Gaceta del congreso",
    "rama": "Rama Legislativa del poder publico",
    "camara": "SENADO DE LA REPÚBLICA",
    "tipo_documento": "Proyecto de Ley",
    "descripcion": "estatutaria No. 201 de 2024 Senado “Proyecto de Ley estatutaria No. 201"
}
